#  Few-shot 프롬프팅

---

## 1. 기본 개념과 환경 설정

### 1.1 Few-shot 프롬프팅이란?

- **Few-shot 프롬프팅**은 AI 모델에게 2-5개의 예시를 제공하여 원하는 작업 패턴을 학습시키는 방법입니다.

- 🎯 **핵심 개념**
    - Zero-shot: 예시 없이 바로 작업 수행
    - One-shot: 1개 예시로 패턴 학습
    - Few-shot: 2-5개 예시로 더 정확한 패턴 학습
    - Dynamic Few-shot: 상황에 맞는 예시를 동적으로 선택


### 1.2 환경 설정

In [1]:
# 기본 라이브러리 설치 및 임포트
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    PromptTemplate, 
    ChatPromptTemplate,
    FewShotPromptTemplate,
    FewShotChatMessagePromptTemplate
)
from langchain_core.output_parsers import StrOutputParser

# 환경 변수 로드
load_dotenv()

# LLM 설정
llm = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0.3
)


---

## 2. Zero-shot vs One-shot vs Few-shot

### 2.1 **Zero-shot** 프롬프팅

- **Zero-shot 프롬프팅**은 예시 없이 AI가 즉시 작업을 수행하는 기법입니다

- 명확한 **지시사항**만으로 원하는 결과를 얻을 수 있어 **사용이 간단**합니다

- 단순하고 직관적인 작업에 적합한 프롬프팅 방식이지만, 작업의 **복잡도에 따라 선택적 사용**이 필요합니다

In [2]:
# 예시 없이 바로 작업 수행
zero_shot_prompt = PromptTemplate(
    input_variables=["topic"],
    template="다음 주제에 대한 핵심 키워드 3개를 추출해주세요: {topic}"
)

chain = zero_shot_prompt | llm | StrOutputParser()
result = chain.invoke({"topic": "인공지능 윤리"})
print(result)

인공지능 윤리의 핵심 키워드 3개는 다음과 같습니다:

1. 책임성 (Accountability)  
2. 공정성 (Fairness)  
3. 투명성 (Transparency)


### 2.2 **One-sho**t 프롬프팅

- **One-shot 프롬프팅**은 하나의 예시를 통해 AI가 작업 패턴을 학습하는 기법입니다

- **Zero-shot** 방식보다 더 나은 성능을 제공하며, **형식화된 작업**에 특히 효과적입니다

- 단일 예시로 **품질 향상**이 가능하나, 해당 예시에 **과의존**할 수 있는 한계가 있습니다


# **One-shot** 프롬프팅



In [10]:
# 하나의 예시로 패턴 학습
one_shot_prompt = PromptTemplate(
    input_variables=["example_input", "example_output", "topic"],
    template="""다음은 주제에서 키워드를 추출하는 예시입니다:

주제: {example_input}
키워드: {example_output}

이제 다음 주제에서 키워드 3개를 추출해주세요:
주제: {topic}
키워드:"""
)

chain = one_shot_prompt | llm | StrOutputParser()
result = chain.invoke({
    "example_input": "기후 변화",
    "example_output": "온실가스,탄소중립,지구온난화",
    "topic": "인공지능 윤리"
})
print(result)

키워드: 책임성, 투명성, 편향성


### 2.3 Few-shot 프롬프팅

- **Few-shot 프롬프팅**은 AI 모델에게 2-5개의 예시를 제공하여 학습시키는 방법입니다

- 이 방식은 **Zero-shot**이나 **One-shot** 프롬프팅보다 더 우수한 성능을 보여주며, 복잡한 작업에서 특히 효과적입니다

- Few-shot 프롬프팅은 높은 성능을 제공하지만, 긴 프롬프트로 인한 **비용 증가**를 고려해야 합니다

In [12]:
# 여러 예시로 더 정확한 패턴 학습
few_shot_prompt = PromptTemplate(
    input_variables=["examples", "topic"],
    template="""다음은 주제에서 키워드를 추출하는 예시들입니다:

{examples}

이제 다음 주제에서 키워드 3개를 추출해주세요. 각 키워드 간에 공백 없음.:
주제: {topic}
키워드:"""
)

examples = """주제: 기후 변화
키워드: 온실가스,탄소 중립,지구 온난화

주제: 블록체인 기술
키워드: 암호화폐,분산원장,스마트 계약 

주제: 메타버스 
키워드: 가상현실,디지털 트윈,NFT"""

chain = few_shot_prompt | llm | StrOutputParser()
result = chain.invoke({
    "examples": examples,
    "topic": "인공지능 방법론"
})
print(result)

키워드: 머신러닝,딥러닝,강화학습


---

## 3. 정적 Few-shot 프롬프팅

### 3.1 **FewShotPromptTemplate** 사용

- FewShotChatMessagePromptTemplate는 LangChain에서 제공하는 템플릿으로, **미리 정의된 고정된 예제들(Fixed Examples)** 을 프롬프트에 포함시켜 모델이 일관된 형식과 품질의 응답을 생성할 수 있도록 돕습니다.

- 이 방식은 특히 특정 형식이나 구조를 가진 출력이 필요한 경우(예: JSON 형식, 특정 분석 리포트 형식 등) 매우 유용하며, 예제들이 고정되어 있어 결과의 일관성을 보장할 수 있습니다.

- 단, 고정된 예제를 사용하기 때문에 상황에 따라 유연하게 대응하기 어려울 수 있으며, 모든 케이스를 커버하기 위해서는 신중한 예제 선택이 필요합니다.

In [14]:
from langchain_core.prompts import FewShotPromptTemplate

# 개별 예시 포맷 정의
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="입력: {input}\n출력: {output}"
)

# 예시 데이터
examples = [
    {"input": "행복한", "output": "슬픈"},
    {"input": "큰", "output": "작은"},
    {"input": "빠른", "output": "느린"},
    {"input": "뜨거운", "output": "차가운"}
]

# Few-shot 프롬프트 템플릿 생성
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="다음 예시들을 참고해서 적절한 답변을 하세요:",
    suffix="입력: {input}\n출력:",
    input_variables=["input"]
)

# 실행
chain = few_shot_prompt | llm | StrOutputParser()
result = chain.invoke({"input": "밝은"})
print(result)

어두운


### 3.2 채팅 모델용 Few-shot 프롬프팅

In [15]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

# 채팅 형태의 예시들
examples = [
    {"input": "2 🦜 2", "output": "4"},
    {"input": "3 🦜 4", "output": "7"},
    {"input": "5 🦜 2", "output": "7"}
]

# 개별 예시 포맷
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("assistant", "{output}")
])

# Few-shot 채팅 프롬프트
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

# 최종 프롬프트
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 새로운 수학 연산 🦜을 이해하는 계산기입니다."),
    few_shot_prompt,
    ("human", "{input}")
])

chain = final_prompt | llm | StrOutputParser()
result = chain.invoke({"input": "7 🦜 3"})
print(result)

10


---

## 4. 동적 Few-shot 프롬프팅


- **Dynamic Few-Shot Prompting**은 상황에 따라 적절한 예시를 동적으로 선택하여 사용하는 고급 프롬프팅 기법으로, **BaseExampleSelector**를 통해 입력값과 가장 연관성이 높은 예시들을 자동으로 선별합니다.

- **example_prompt**를 통해 선택된 예시들을 AI 시스템이 이해하기 쉬운 형태(예: human-AI 대화 , human-function call)로 변환하여 더 효과적인 학습과 응답 생성이 가능하게 합니다.


- **장점**

    - 상황에 맞는 가장 연관성 높은 예시만을 선택적으로 활용할 수 있다
    - 프롬프트의 길이를 효율적으로 관리할 수 있다
    - 응답의 일관성과 품질을 향상시킬 수 있다

### 4.1 **SemanticSimilarityExampleSelector**

- 의미적 유사도를 기반으로 예시를 선택하며, 이를 통해 주어진 입력 상황에 가장 적합한 예시들만을 효율적으로 활용할 수 있습니다.


In [16]:
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# 고객 서비스 예시 데이터
examples = [
    {
        "input": "환불 절차가 어떻게 되나요?",
        "output": "환불 신청 → 상품 반송 → 검수 후 3-5일 내 환불 완료됩니다."
    },
    {
        "input": "배송이 늦어지고 있어요",
        "output": "주문번호를 알려주시면 배송 상태를 즉시 확인해드리겠습니다."
    },
    {
        "input": "제품이 불량이에요",
        "output": "불량 부분 사진과 함께 1:1 문의 접수해주시면 빠르게 처리해드리겠습니다."
    },
    {
        "input": "사이즈가 안 맞아요",
        "output": "사이즈 교환은 무료로 진행됩니다. 다른 사이즈로 교환 발송해드리겠습니다."
    }
]

# 유사도 기반 예시 선택기 생성
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(model="text-embedding-3-small"),
    Chroma,
    k=2  # 상위 2개 예시 선택
)

# 동적 Few-shot 프롬프트 생성
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=PromptTemplate.from_template(
        "고객: {input}\n상담원: {output}"
    ),
    prefix="다음은 고객 서비스 상담 예시들입니다:",
    suffix="고객: {input}\n상담원:",
    input_variables=["input"]
)

# 실행
chain = dynamic_prompt | llm | StrOutputParser()
result = chain.invoke({"input": "상품이 파손되어 도착했어요"})
print(result)

상담원: 불편을 드려 죄송합니다. 파손된 부분 사진과 함께 1:1 문의를 접수해주시면 신속히 교환 또는 환불 절차를 안내해드리겠습니다.


### 4.2 MMR 기반 선택


In [17]:
from langchain_core.example_selectors import MaxMarginalRelevanceExampleSelector
from langchain_community.vectorstores import FAISS

# MMR 기반 예시 선택기
mmr_example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(model="text-embedding-3-small"),
    FAISS,
    k=2
)

mmr_prompt = FewShotPromptTemplate(
    example_selector=mmr_example_selector,
    example_prompt=PromptTemplate.from_template(
        "고객: {input}\n상담원: {output}"
    ),
    prefix="다음은 고객 서비스 상담 예시들입니다:",
    suffix="고객: {input}\n상담원:",
    input_variables=["input"]
)

chain = mmr_prompt | llm | StrOutputParser()
result = chain.invoke({"input": "주문 취소하고 싶어요"})
print(result)

상담원: 주문 취소를 도와드리겠습니다. 주문번호를 알려주시면 확인 후 처리해드리겠습니다. 단, 이미 배송 준비 중인 경우 취소가 어려울 수 있는 점 양해 부탁드립니다.


---

## 5. 실전 활용 사례

### 5.1 뉴스 기사 감정 분석

In [18]:
sentiment_examples = [
    {
        "input": "새로운 치료법 개발로 환자들에게 희망을 주고 있습니다.",
        "output": "긍정적 | 희망적 | 의료혁신"
    },
    {
        "input": "경제 불황으로 많은 기업들이 어려움을 겪고 있습니다.",
        "output": "부정적 | 우려 | 경제위기"
    },
    {
        "input": "AI 기술 발전이 일자리에 미칠 영향에 대해 논의가 필요합니다.",
        "output": "중립적 | 신중함 | 기술변화"
    }
]

sentiment_prompt = FewShotPromptTemplate(
    examples=sentiment_examples,
    example_prompt=PromptTemplate.from_template(
        "뉴스: {input}\n감정분석: {output}"
    ),
    prefix="다음은 뉴스 기사 감정 분석 예시들입니다:",
    suffix="뉴스: {input}\n감정분석:",
    input_variables=["input"]
)

chain = sentiment_prompt | llm | StrOutputParser()
result = chain.invoke({
    "input": "정부의 새로운 환경 정책이 발표되어 기업들의 반응이 엇갈리고 있습니다."
})
print(result)

감정분석: 중립적 | 신중함 | 환경정책


### 5.2 코드 설명 생성


In [19]:
code_examples = [
    {
        "input": "def fibonacci(n):\n    if n <= 1:\n        return n\n    return fibonacci(n-1) + fibonacci(n-2)",
        "output": "피보나치 수열의 n번째 값을 재귀적으로 계산하는 함수입니다. 기저조건은 n이 1 이하일 때이며, 그렇지 않으면 이전 두 값의 합을 반환합니다."
    },
    {
        "input": "numbers = [1, 2, 3, 4, 5]\nsquared = [x**2 for x in numbers]",
        "output": "리스트의 각 원소를 제곱하여 새로운 리스트를 만드는 리스트 컴프리헨션입니다. 결과는 [1, 4, 9, 16, 25]가 됩니다."
    }
]

code_prompt = FewShotPromptTemplate(
    examples=code_examples,
    example_prompt=PromptTemplate.from_template(
        "코드:\n{input}\n\n설명: {output}"
    ),
    prefix="다음은 코드 설명 예시들입니다:",
    suffix="코드:\n{input}\n\n설명:",
    input_variables=["input"]
)

chain = code_prompt | llm | StrOutputParser()
result = chain.invoke({
    "input": "def binary_search(arr, target):\n    left, right = 0, len(arr) - 1\n    while left <= right:\n        mid = (left + right) // 2\n        if arr[mid] == target:\n            return mid\n        elif arr[mid] < target:\n            left = mid + 1\n        else:\n            right = mid - 1\n    return -1"
})
print(result)

이진 탐색 알고리즘을 사용하여 정렬된 배열에서 특정 값(target)의 인덱스를 찾는 함수입니다. 배열의 중간값과 target을 비교하며 탐색 범위를 절반씩 좁혀가고, 값을 찾으면 해당 인덱스를 반환하며, 찾지 못하면 -1을 반환합니다.


---

## 6. 연습 문제 ~ 09:50분

### 🎯 문제 1: 기본 Few-shot 프롬프팅

**요구사항**: 제품 리뷰를 입력받아 별점(1-5점)을 예측하는 Few-shot 프롬프트를 작성하세요.

**예시 데이터**:
- "정말 만족스러운 제품입니다. 품질이 훌륭해요!" → 5점
- "가격 대비 괜찮은 것 같습니다." → 3점
- "배송이 늦고 포장이 엉망이었어요." → 2점

In [ ]:
# 여기에 코드를 작성하세요

### 🎯 문제 2: 동적 Few-shot 프롬프팅

**요구사항**: 다양한 프로그래밍 언어의 코딩 질문에 대해 적절한 예시를 동적으로 선택하여 답변하는 시스템을 구현하세요.

**예시 데이터**:
- Python 리스트 조작, Java 클래스 설계, JavaScript 비동기 처리 등

In [ ]:
# 여기에 코드를 작성하세요

### 🎯 문제 3: 복합 Few-shot 프롬프팅

**요구사항**: 이메일을 입력받아 다음을 분류하는 시스템을 만드세요:
1. 카테고리 (업무/개인/스팸)
2. 우선순위 (높음/보통/낮음)
3. 감정 톤 (긍정/중립/부정)


In [ ]:
# 여기에 코드를 작성하세요

---

## 📚 추가 학습 자료

### ✅ 성공적인 Few-shot 프롬프팅을 위한 팁

1. **명확한 예시 구조**: 입력과 출력이 명확히 구분되도록 포맷팅
2. **다양성 있는 예시**: 서로 다른 패턴을 보여주는 예시 선택
3. **일관된 품질**: 모든 예시가 원하는 수준의 품질을 유지
4. **적절한 개수**: 2-5개의 예시가 일반적으로 최적
5. **동적 선택 활용**: 입력에 맞는 관련성 높은 예시 자동 선택

### ⚠️ 주의사항

- 너무 많은 예시는 토큰 비용 증가와 성능 저하 야기
- 편향된 예시는 모델의 편향된 응답 유도
- 복잡한 태스크는 Chain-of-Thought와 결합 고려


### 🔗 유용한 링크

- [LangChain 공식 문서 - Few-shot Examples](https://python.langchain.com/docs/how_to/few_shot_examples/)
- [Example Selectors 가이드](https://python.langchain.com/docs/how_to/example_selectors/)

---

In [ ]:
# =================================================================================
# 연습문제 1 해답: 제품 리뷰 별점 예측
# =================================================================================


# 별점 예측 예시 데이터
rating_examples = [
    {
        "input": "정말 만족스러운 제품입니다. 품질이 훌륭해요!",
        "output": "5점"
    },
    {
        "input": "가격 대비 괜찮은 것 같습니다.",
        "output": "3점"
    },
    {
        "input": "배송이 늦고 포장이 엉망이었어요.",
        "output": "2점"
    },
    {
        "input": "기대했던 것보다 훨씬 좋네요. 강력 추천합니다!",
        "output": "5점"
    },
    {
        "input": "그냥 그래요. 특별할 건 없지만 나쁘지도 않아요.",
        "output": "3점"
    }
]

# Few-shot 프롬프트 생성
rating_prompt = FewShotPromptTemplate(
    examples=rating_examples,
    example_prompt=PromptTemplate.from_template(
        "리뷰: {input}\n별점: {output}"
    ),
    prefix="다음은 제품 리뷰에 따른 별점 예측 예시들입니다:",
    suffix="리뷰: {input}\n별점:",
    input_variables=["input"]
)

# 테스트 실행
chain = rating_prompt | llm | StrOutputParser()
test_review = "배송은 빨랐지만 제품 품질이 기대에 못 미쳤어요."
result = chain.invoke({"input": test_review})
print(f"테스트 리뷰: {test_review}")
print(f"예측 별점: {result}")

In [ ]:

# =================================================================================
# 연습문제 2 해답: 동적 프로그래밍 Q&A 시스템
# =================================================================================


# 프로그래밍 Q&A 예시 데이터
programming_examples = [
    {
        "input": "Python에서 리스트의 마지막 요소를 제거하는 방법",
        "output": "pop() 메서드를 사용하면 됩니다. 예: my_list.pop() 또는 del my_list[-1]을 사용할 수 있습니다."
    },
    {
        "input": "Java에서 문자열을 정수로 변환하는 방법",
        "output": "Integer.parseInt() 메서드를 사용합니다. 예: int num = Integer.parseInt(\"123\");"
    },
    {
        "input": "JavaScript에서 배열에 요소 추가하는 방법",
        "output": "push() 메서드를 사용합니다. 예: array.push('new element') 또는 spread 연산자 [...array, 'new element']를 사용할 수 있습니다."
    },
    {
        "input": "Python에서 딕셔너리 키가 존재하는지 확인하는 방법",
        "output": "in 연산자를 사용합니다. 예: if 'key' in my_dict: 또는 my_dict.get('key') is not None을 사용할 수 있습니다."
    },
    {
        "input": "Java에서 ArrayList 크기 구하는 방법",
        "output": "size() 메서드를 사용합니다. 예: int size = arrayList.size();"
    }
]

# 동적 예시 선택기 생성
programming_selector = SemanticSimilarityExampleSelector.from_examples(
    programming_examples,
    OpenAIEmbeddings(model="text-embedding-3-small"),
    Chroma,
    k=2  # 가장 관련성 높은 2개 예시 선택
)

# 동적 Few-shot 프롬프트 생성
programming_prompt = FewShotPromptTemplate(
    example_selector=programming_selector,
    example_prompt=PromptTemplate.from_template(
        "질문: {input}\n답변: {output}"
    ),
    prefix="다음은 프로그래밍 관련 질문과 답변 예시들입니다:",
    suffix="질문: {input}\n답변:",
    input_variables=["input"]
)

# 테스트 실행
chain = programming_prompt | llm | StrOutputParser()
test_question = "Python에서 리스트를 정렬하는 방법"
result = chain.invoke({"input": test_question})
print(f"질문: {test_question}")
print(f"답변: {result}")


In [ ]:
# =================================================================================
# 연습문제 3 해답: 복합 이메일 분류 시스템
# =================================================================================

# 이메일 분류 예시 데이터
email_examples = [
    {
        "input": "안녕하세요. 내일 회의 일정 변경 요청드립니다. 급한 사안이 생겨서 오후 3시로 변경 가능할까요?",
        "output": "카테고리: 업무 | 우선순위: 높음 | 감정: 중립"
    },
    {
        "input": "생일 파티 초대합니다! 이번 주말 저희 집에서 작은 모임 있어요. 시간 되시면 꼭 오세요~",
        "output": "카테고리: 개인 | 우선순위: 낮음 | 감정: 긍정"
    },
    {
        "input": "축하합니다! 당신이 100만원에 당첨되었습니다. 지금 바로 링크를 클릭하세요!",
        "output": "카테고리: 스팸 | 우선순위: 낮음 | 감정: 중립"
    },
    {
        "input": "프로젝트 마감이 내일인데 아직 검토가 완료되지 않았습니다. 빠른 검토 부탁드립니다.",
        "output": "카테고리: 업무 | 우선순위: 높음 | 감정: 부정"
    },
    {
        "input": "새로운 카페 오픈했어요! 첫 주 할인 이벤트 중이니 한번 놀러 오세요.",
        "output": "카테고리: 개인 | 우선순위: 낮음 | 감정: 긍정"
    }
]

# 이메일 분류 프롬프트 생성
email_prompt = FewShotPromptTemplate(
    examples=email_examples,
    example_prompt=PromptTemplate.from_template(
        "이메일: {input}\n분류: {output}"
    ),
    prefix="다음은 이메일 분류 예시들입니다:",
    suffix="이메일: {input}\n분류:",
    input_variables=["input"]
)

# 테스트 실행
chain = email_prompt | llm | StrOutputParser()
test_email = "시스템 장애로 인해 서비스가 중단되었습니다. 긴급 복구 작업이 필요합니다."
result = chain.invoke({"input": test_email})
print(f"테스트 이메일: {test_email}")
print(f"분류 결과: {result}")

---